In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson,skellam
from scipy.optimize import minimize

n=np.linspace(1,190,190)
epl = pd.read_csv("http://www.football-data.co.uk/mmz4281/2021/I1.csv",skiprows=n)
epl = epl[['HomeTeam','AwayTeam','FTHG','FTAG']]
epl = epl.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})
#epl.head()

#pd.set_option("max_rows", None)
epl_new = pd.read_csv("http://www.football-data.co.uk/mmz4281/2122/I1.csv")
epl_new = epl_new[['HomeTeam','AwayTeam','FTHG','FTAG']]
epl_new = epl_new.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})
#epl_new.head()

epl = pd.concat([epl,epl_new])
epl.index = range(epl.shape[0])
epl

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

def solve_parameters(dataset, debug = False, init_vals=None, options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['HomeTeam'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['AwayTeam'].unique())
    #if not np.array_equal(teams, away_teams):
    #    raise ValueError("Something's not right")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0, 1.0]) # rho (score correction), gamma (home advantage)
                                     ))
    def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
        return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
                np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        log_like = [dc_log_like(row.HomeGoals, row.AwayGoals, score_coefs[row.HomeTeam], defend_coefs[row.HomeTeam],
                     score_coefs[row.AwayTeam], defend_coefs[row.AwayTeam], rho, gamma) for row in dataset.itertuples()]
        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints, **kwargs)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))

def calc_means(param_dict, homeTeam, awayTeam):
    return [np.exp(param_dict['attack_'+homeTeam] + param_dict['defence_'+awayTeam] + param_dict['home_adv']),
            np.exp(param_dict['defence_'+homeTeam] + param_dict['attack_'+awayTeam])]

def dixon_coles_simulate_match(params_dict, homeTeam, awayTeam, max_goals=10):
    team_avgs = calc_means(params_dict, homeTeam, awayTeam)
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in team_avgs]
    output_matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    correction_matrix = np.array([[rho_correction(home_goals, away_goals, team_avgs[0],
                                                   team_avgs[1], params['rho']) for away_goals in range(2)]
                                   for home_goals in range(2)])
    output_matrix[:2,:2] = output_matrix[:2,:2] * correction_matrix
    return output_matrix
    
params = solve_parameters(epl)



<ipython-input-7-ea06f91d86f3>:58: RuntimeWarning: divide by zero encountered in log
  np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))
<ipython-input-7-ea06f91d86f3>:57: RuntimeWarning: invalid value encountered in log
  return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) +


In [6]:

import os

file = open("seriea.txt", "w")

#loading the list of matches
m=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRcyK3wrZDe-DyL1vYh8RO-r9CIer3LqBxqJZNM7Wf6LGj9AuFUh_uc4zPd-LdNfuilCeWumUxnTRZ-/pub?output=csv',header=None)
#convert to a DataFrame
m = pd.DataFrame(m)

n_matchs = len(m.index) #number of matches 
g = 10;   #goals to take into account

for i in range(n_matchs):
    
    ht = m.loc[i,0]
    at = m.loc[i,1]
    match =  dixon_coles_simulate_match(params, ht, at, g)  #matrix with match-score results

    M=match

    d = np.trace(match)*100;        #draw

    s=0;
    for i in range(1,g):
        for j in range(0,i):
            s=s+M[i][j]
            #print (M[i][j])
    hw=s*100;                      #home win

    s=0;
    for i in range(0,g):
        for j in range(i+1,g):
            s=s+M[i][j]
            #print (M[i][j])
    aw=s*100;                      #away win
    hw = "{:.2f}".format(hw)
    d =  "{:.2f}".format(d)
    aw = "{:.2f}".format(aw)
    
    print (ht,': ',hw)
    print (at,': ',aw)
    print ('draw : ',d);
    print ('')
    file.write(ht + ': ' + hw + os.linesep)
    file.write(at + ': ' + aw + os.linesep)
    file.write('draw : '+ d + os.linesep)
    file.write(' ' + os.linesep)
    
file.close()   


Spezia :  12.54
Milan :  70.27
draw :  17.17

Inter :  65.99
Atalanta :  14.92
draw :  19.08

Genoa :  52.43
Verona :  24.67
draw :  22.90

Juventus :  59.49
Sampdoria :  17.00
draw :  23.51

Udinese :  34.27
Fiorentina :  40.24
draw :  25.48

Sassuolo :  85.47
Salernitana :  4.86
draw :  9.48



KeyError: 'defence_Bolonia'